# Extract Historical Data From PurpleAir Sensors
In this notebook, we will extract historical data from PurpleAir sensors in the Ukranian Village neighborhood of Chicago. To account for seasonal variability, we wanted to collect a 3 week summer dataset (1st-20th July 2021) and a 3 week winter dataset (1st - 20th Jan 2021). In order to collect historical data, we first have to use the PurpleAir API to get the Thingspeak API keys and sensor IDs. We then pull the data from Thingspeak. Unfortunately, Thingspeak has a date limit and does not keep data older than about 3 1/2 months. For this reason, we will pull the oldest and newest data possible.

In [1]:
# load libraries
import os
os.chdir('../') # move up one directory level

from datetime import datetime
import pandas as pd

from src.config import PURPLE_AIR_KEY # this is your personal Purple Air Key (see README)
from src.data import pull_purpleair_data, pull_purpleair_historical, add_pa_sensor

In [2]:
# pull list of PurpleAir sensors from JSON file
sensor_df = pd.read_json("data/pa_sensors.json")

## Update PurpleAir sensor file
1. Visit [map.purpleair.com](https://map.purpleair.com) and click on the sensor of interest.
1. Grab the Purple Air sensor ID from the url (eg. <span style="font-size:11px">map.purpleair.com/1/m/i/mAQI/a10/p604800/cC0?select=</span><b>87741</b><span style="font-size:11px">#12.6/41.87991/-87.67404)</span>
1. Add the Sensor ID(s), Neighborhood, and City to the `pa_sensors.json` file using the following code.

In [3]:
# neighborhood where sensors are located
AREA = 'Ukranian Village' # update with neighborhood of interest, user-defined

# list of PurpleAir sensor IDs in the neighborhood
ID_LIST = [87741,3499] # update with sensors of interest, pulled from map.purpleair.com

# name of JSON file containing sensor info
JSON_FILE = 'data/pa_sensors.json'

# city where sensors are located
CITY = 'Chicago'

# year that data pull begins
YEAR = 2021

In [4]:
#add sensors
for sensor in ID_LIST:
    sensor_df = add_pa_sensor(
        sensor_df, 
        sensor_id=sensor, 
        neighborhood=AREA,
        city=CITY
    )

## Create a list of Thingspeak IDs and keys
Run the code below to get the Thingspeak data and update the sensor file

In [5]:
# create dataframe for sensors data
pa_sensor_data = pull_purpleair_data(
    sensors = sensor_df, 
    city = CITY,
    neighborhood = AREA,
    key = PURPLE_AIR_KEY
)

In [6]:
# update sensor_df with Thingspeak data
for sensor in pa_sensor_data[pa_sensor_data.DEVICE_LOCATIONTYPE == 'outside']['ID']:
    if pd.isnull(sensor_df[(sensor_df.SensorID == sensor)]['ThingspeakID']).values[0]:
        # get ThingspeakID
        sensor_df.loc[
            sensor_df.SensorID == sensor,'ThingspeakID'
        ] = int(pa_sensor_data[pa_sensor_data.ID == sensor]['THINGSPEAK_PRIMARY_ID'].values[0])
        
        # get ThingspeakKey
        sensor_df.loc[
            sensor_df.SensorID == sensor,'ThingspeakKey'
        ] = pa_sensor_data[pa_sensor_data.ID == sensor]['THINGSPEAK_PRIMARY_ID_READ_KEY'].values[0]
        
        # get sensor Location
        sensor_df.loc[
            sensor_df.SensorID == sensor,'Location'
        ] = pa_sensor_data[pa_sensor_data.ID == sensor]['Label'].values[0]
        
        # get ThingspeakID for child sensor
        sensor_df.loc[
            sensor_df.SensorID == sensor,'ThingspeakID_B'
        ] = int(pa_sensor_data[pa_sensor_data.ID == sensor]['THINGSPEAK_SECONDARY_ID'].values[0])
        
        # get ThingspeakKey for child sensor
        sensor_df.loc[
            sensor_df.SensorID == sensor,'ThingspeakKey_B'
        ] = pa_sensor_data[pa_sensor_data.ID == sensor]['THINGSPEAK_SECONDARY_ID_READ_KEY'].values[0]

In [7]:
# check that data was updated
sensor_df

,SensorID,Neighborhood,City,ThingspeakID,ThingspeakKey,Location,ThingspeakID_B,ThingspeakKey_B
0,96035,McKinley Park,Chicago,1267967.0,6AB8IRLJJPKIFME0,36th and Paulina,1267969.0,L3Y8P2IWC9NY4HLO
1,96395,McKinley Park,Chicago,1271673.0,5REK7K675FMHLXCU,38th & Winchester,1271675.0,324MUY4V609SF5M0
2,94975,McKinley Park,Chicago,1261965.0,UKYER4XQP4R4KLVQ,39th and Damen (NLEI),1261967.0,YUSL71LBU2F7DJAJ
3,113494,South Lawndale,Chicago,1440112.0,9O7I4C3E4GZ46WWX,LVEJO,1440126.0,SNCNY2YTNI3YP0RN
4,113474,South Lawndale,Chicago,1440052.0,5NMUSWFVMU48KQLT,LVEJO_3,1440054.0,MW4CZO5RL3X0MHBA
5,5774,South Lawndale,Chicago,395514.0,9WOL7PCGSU9ZV8P5,LVEJO_5,395517.0,2TDGMCMLRQPM745H
6,87741,Ukranian Village,Chicago,1215908.0,KP43J7MJGIHV457E,Ukrainian Village Chicago,1215910.0,N3YVGO0HGNQN6VTI
7,3499,Ukranian Village,Chicago,334460.0,39XP408ZUPEBCDG9,Ukrainian Village,334461.0,ZHCH54TYCMYF4RPU


In [8]:
# save dataframe to json file
sensor_df.to_json(JSON_FILE, indent=2)

In [9]:
# create a dictionary of parent Thingspeaks IDs and keys
id_dict = {}
for idx in pa_sensor_data[~pa_sensor_data.Label.str.endswith('B')].index:
    id_dict[pa_sensor_data.loc[idx,'ID']]= {
        'ThingspeakID': pa_sensor_data.loc[idx,'THINGSPEAK_PRIMARY_ID'],
        'ThingspeakKey': pa_sensor_data.loc[idx,'THINGSPEAK_PRIMARY_ID_READ_KEY'],
        'Location': pa_sensor_data.loc[idx,'Label']
    }

In [10]:
# create a dictionary of child purpleair sensors
id_dict_b = {}
for idx in pa_sensor_data[pa_sensor_data.Label.str.endswith('B')].index:
    id_dict_b[pa_sensor_data.loc[idx,'ID']]= {
        'ThingspeakID_B': pa_sensor_data.loc[idx,'THINGSPEAK_PRIMARY_ID'],
        'ThingspeakKey_B': pa_sensor_data.loc[idx,'THINGSPEAK_PRIMARY_ID_READ_KEY']
    }

## Pull June PurpleAir Data

In [11]:
# define Thingspeak column labels
PARENT_PRIMARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': 'PM1.0 (CF=1) ug/m3',
    'field2': 'PM2.5 (CF=1) ug/m3',
    'field3': 'PM10.0 (CF=1) ug/m3',
    'field4': 'UptimeMinutes',
    'field5': 'ADC',
    'field6': 'Temperature_F',
    'field7': 'Humidity_%',
    'field8': 'PM2.5 (CF=ATM) ug/m3',
}

PARENT_SECONDARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': '0.3um/dl',
    'field2': '0.5um/dl',
    'field3': '1.0um/dl',
    'field4': '2.5um/dl',
    'field5': '5.0um/dl',
    'field6': '10.0um/dl',
    'field7': 'PM1.0 (CF=ATM) ug/m3',
    'field8': 'PM10 (CF=ATM) ug/m3',
}

CHILD_PRIMARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': 'PM1.0 (CF=1) ug/m3',
    'field2': 'PM2.5 (CF=1) ug/m3',
    'field3': 'PM10.0 (CF=1) ug/m3',
    'field4': 'UptimeMinutes',
    'field5': 'RSSI_dbm',
    'field6': 'Atmospheric Pressure',
    'field7': 'gas_sensor',
    'field8': 'PM2.5 (CF=ATM) ug/m3',
}

CHILD_SECONDARY_COLS = {
    'created_at': 'created_at',
    'entry_id': 'entry_id',
    'field1': '0.3um/dl',
    'field2': '0.5um/dl',
    'field3': '1.0um/dl',
    'field4': '2.5um/dl',
    'field5': '5.0um/dl',
    'field6': '10.0um/dl',
    'field7': 'PM1.0 (CF=ATM) ug/m3',
    'field8': 'PM10 (CF=ATM) ug/m3'
}

In [12]:
# create PurpleAir sensor dataframe
pa_sensor_df = pd.DataFrame(columns=PARENT_PRIMARY_COLS.values())
pa_sensor_df['sensor'] = ''

# iterate through primary sensor data
for idx,sensor in enumerate(sensor_df[sensor_df.Neighborhood == AREA].SensorID):
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=int(sensor_df.loc[idx,'ThingspeakID']),
        key=sensor_df.loc[idx,'ThingspeakKey'],
        col_names=PARENT_PRIMARY_COLS,
        start_date=datetime(YEAR, 8, 1, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_df = pd.concat([pa_sensor_df, df])

In [13]:
days = pa_sensor_df.created_at.dt.day.nunique()
print(f'There are {days} days of sensor data.')

There are 28 days of sensor data.


In [18]:
pa_sensor_df.to_csv(f"data/summer_{str(YEAR)}_pa_parent_{AREA.lower().replace(' ','_')}.csv")

In [19]:
# create PurpleAir sensor B dataframe
pa_sensor_b_df = pd.DataFrame(columns=CHILD_PRIMARY_COLS.values())
pa_sensor_b_df['sensor'] = ''

# iterate through primary sensor data
for sensor in id_dict_b.keys():
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=id_dict_b[sensor]['ThingspeakID_B'],
        key=id_dict_b[sensor]['ThingspeakKey_B'],
        col_names=CHILD_PRIMARY_COLS,
        start_date=datetime(YEAR, 8, 1, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_b_df = pd.concat([pa_sensor_b_df, df])

In [20]:
pa_sensor_b_df.to_csv(f"data/summer_{str(YEAR)}_pa_child_{AREA.lower().replace(' ','_')}.csv")

# Pull Autumn 2021 Data

In [21]:
# create PurpleAir sensor dataframe
pa_sensor_df2 = pd.DataFrame(columns=PARENT_PRIMARY_COLS.values())
pa_sensor_df2['sensor'] = ''

# iterate through primary sensor data
for sensor in id_dict.keys():
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=id_dict[sensor]['ThingspeakID'],
        key=id_dict[sensor]['ThingspeakKey'],
        col_names=PARENT_PRIMARY_COLS,
        start_date=datetime(YEAR, 11, 2, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_df2 = pd.concat([pa_sensor_df2, df])

In [22]:
days2 = pa_sensor_df2.created_at.dt.day.nunique()
print(f'There are {days2} days of sensor data.')

There are 28 days of sensor data.


In [23]:
pa_sensor_df2.to_csv(f"data/autumn_{str(YEAR)}_pa_parent_{AREA.lower().replace(' ','_')}.csv")

In [24]:
# create PurpleAir sensor B dataframe
pa_sensor_b_df2 = pd.DataFrame(columns=CHILD_PRIMARY_COLS.values())
pa_sensor_b_df2['sensor'] = ''

# iterate through primary sensor data
for sensor in id_dict_b.keys():
    # pull sensor data
    df = pull_purpleair_historical(
        weeks_to_get=3,
        channel=id_dict_b[sensor]['ThingspeakID_B'],
        key=id_dict_b[sensor]['ThingspeakKey_B'],
        col_names=CHILD_PRIMARY_COLS,
        start_date=datetime(YEAR, 11, 2, 21, 32, 15, 259245)
    )
    df['sensor'] = sensor
    
    # append to pa_sensor_df
    pa_sensor_b_df2 = pd.concat([pa_sensor_b_df2, df])

In [25]:
pa_sensor_b_df2.to_csv(f"data/autumn_{str(YEAR)}_pa_child_{AREA.lower().replace(' ','_')}.csv")